First make sure everything is installed ...

In [18]:
# Install drawing library
!pip3 install Pillow

# Install simulation library
!pip3 install pybullet

# Install video library
!pip3 install opencv-python

# Install some widgets
!pip3 install ipywidgets
!pip3 install jupyterlab_widgets

!pip3 install moviepy


Import everything we'll need for this notebook

In [29]:

import pybullet as p
import pybullet_data
import math
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
import numpy as np
import random
#from tqdm import tqdm
from tqdm.notebook import trange, tqdm

from IPython.display import display
#from IPython.display import Image

import cv2 as cv
fourcc = cv.VideoWriter_fourcc(*'mp4v')
#fourcc = cv.VideoWriter_fourcc(*'MJPG')

import os
from google.colab import drive

# Connect to Google Drive
drive.mount('/content/gdrive')

# Go to the root drive folder
os.chdir('/content/gdrive/MyDrive/INASCON2021')


Define some useful classes:

Joint - helper

ScaledCanvas - helper

Smarticle - controls each bot

In [20]:
# First let's define a class for the JointInfo. (will be used by the simulation below)

from dataclasses import dataclass

# This data class stores information about a joint
@dataclass
class Joint:
    index: int
    name: str
    type: int
    gIndex: int
    uIndex: int
    flags: int
    damping: float
    friction: float
    lowerLimit: float
    upperLimit: float
    maxForce: float
    maxVelocity: float
    linkName: str
    axis: tuple
    parentFramePosition: tuple
    parentFrameOrientation: tuple
    parentIndex: int

    def __post_init__(self):
        self.name = str(self.name, 'utf-8')
        self.linkName = str(self.linkName, 'utf-8')


In [21]:
# Define a helper class to aid with drawing (simulation world to pixel conversion)

class ScaledCanvas:
    def __init__(self, wh, box):
        #self.image = Image.new("L", wh) # grayscale
        self.image = Image.new("RGB", wh)  # RGB
        self.draw = ImageDraw.Draw(self.image)
        self.box = box

    #def polygon(self, points, fill=255):
    def polygon(self, points, fill = (150, 150, 150)):
        points = list(points)
        for i in range(len(points)):
            points[i] = ((points[i][0] - self.box[0]) * (self.image.width / self.box[2]), (points[i][1] - self.box[1]) * (self.image.height / self.box[3]))

        self.draw.polygon(points, fill = fill)



In [22]:
# Define a Smarticle class
# This class handles the behavior of each bot in the simulation
class Smarticle:
    index: int # ID of the bot in the simulation engine
    j1: Joint # arm joints
    j2: Joint

    # Physical parameters for actuation of the arms
    max_torque: float
    max_speed: float
        
        

    def __init__(self, index):
        self.index = index
        
        # default torque and angular speed
        self.max_torque = 2.5
        self.max_speed = 6 * math.pi  # per sec
        
        self.j1 = Joint(*p.getJointInfo(index, 0))
        self.j2 = Joint(*p.getJointInfo(index, 1))

        # add damping such that the angular speed is the terminal speed
        damping = self.max_torque / self.max_speed
        #p.changeDynamics(index, -1, jointDamping = damping)
        p.changeDynamics(index, self.j1.index, jointDamping = damping)
        p.changeDynamics(index, self.j2.index, jointDamping = damping)

    # Operate a given joint - set it to target an angular position
    def OperateArm(self, joint, targetAngle):
        p.setJointMotorControl2(
            bodyIndex=self.index,
            jointIndex=joint.index,
            controlMode=p.POSITION_CONTROL,
            targetPosition=targetAngle,
            targetVelocity=self.max_speed,
            maxVelocity=self.max_speed,
            force=self.max_torque)

    def Gait(self, t):
        phase = t % 1.0

        # (0, 0) is straight arms (co-linear with body)
        # (math.pi/2, math.pi/2) is arms at 90 degrees, towards opposing directions
        # (math.pi/2, -math.pi/2) or (-math.pi/2, math.pi/2) are C-shaped configurations

        # Replace this by your own gait logic, determined by "phase" (between 0 and 1)
        return (random.uniform(-math.pi/2, math.pi/2), random.uniform(-math.pi/2, math.pi/2))

    def OperateSmarticle(self, t):
        # Check bot's velocity to see if it is flipping or flying off (it's not supposed to!)
        velocity = p.getBaseVelocity(self.index)
        angularVelocity = velocity[1]
        velocity = velocity[0]

        # Is the bot flipping over or flying off?
        if velocity[2] > 1e-3 or angularVelocity[0] > 1e-6 or angularVelocity[1] > 1e-6:
            velocity = (velocity[0], velocity[1], 0)
            angularVelocity = (0, 0, angularVelocity[2])
            # Reset z-direction velocity and x/y-direction rotation
            p.resetBaseVelocity(self.index, velocity, angularVelocity)

        # Get target arm angles
        (a1, a2) = self.Gait(t)

        # Set servos to given angles
        self.OperateArm(self.j1, a1)
        self.OperateArm(self.j2, a2)

        # Add diffusion in the form of force noise
        D = 10
        A = 0.5

        force_noise = (random.gauss(0, D), random.gauss(0, D), 0)
        p.applyExternalForce(self.index, -1, force_noise, (0, 0, 0), p.LINK_FRAME)
        
        # Set the adjusted velocities
        p.resetBaseVelocity(self.index, velocity, angularVelocity)


In [23]:
N = 1 # number of bots
box_size = 1 # box size (box_size x box_size)

dt = 1e-3 # this will be the time-step

recording_prefix = '' # set this to some name to record the trajectory and video
record_every = 10 # record every given number of steps

# Set total simulation time
sim_time = 1

# length of simulation, in number of steps (sim_length * dt is the simulation time)
sim_length = int(math.ceil(sim_time / dt))

# Size (image_size x image_size) of image generated in the video
image_size = 2048
#image_size = 1024
#image_size = 512


if len(recording_prefix) > 0:
    trajectory_file_name = recording_prefix + '.txt'
    video_file_name = recording_prefix + '.avi'
else:
    trajectory_file_name = ''
    video_file_name = ''


# Seed the random number generator (change to generate a different instance of the simulation)
random.seed(1) 



# Connect to simulation engine
p.connect(p.DIRECT)

# Adjust simulation engine data path
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Reset the simulation engine
p.resetSimulation()



# Create walls (box_size x box_size) centered around (0, 0) 

w = p.createCollisionShape(p.GEOM_PLANE, planeNormal=(1,0,0))
p.createMultiBody(0, w, basePosition=(-0.5 * box_size, 0, 0))

w = p.createCollisionShape(p.GEOM_PLANE, planeNormal=(-1,0,0))
p.createMultiBody(0, w, basePosition=(0.5 * box_size, 0, 0))

w = p.createCollisionShape(p.GEOM_PLANE, planeNormal=(0,1,0))
p.createMultiBody(0, w, basePosition=(0, -0.5 * box_size, 0))

w = p.createCollisionShape(p.GEOM_PLANE, planeNormal=(0,-1,0))
p.createMultiBody(0, w, basePosition=(0, 0.5 * box_size, 0))


# initialize smarticles list
smarticles = []

# reset time
time = 0

# Change this if you want a condensed initial configuration
creation_region = box_size
# creation_region = box_size * 0.5

num_attempts = 0

# Load N Smarticle bots
for i in range(N):

    if i > 0:
        # Generate random position and rotation
        pos = [creation_region * random.random() - creation_region * 0.5, \
               creation_region * random.random() - creation_region * 0.5, 0]
        qtr = p.getQuaternionFromEuler((0, 0, random.random() * math.pi))
    else:
        # Always place the first one at 0,0 with orientation 0
        pos = [0, 0, 0]
        qtr = p.getQuaternionFromEuler((0, 0, 0))
    
    # Load smarticle onto the given position and rotation
    s = p.loadURDF('SmarticleLowProfile.urdf', pos, qtr)
    
    s = Smarticle(s) # Create managing object
    smarticles.append(s) # Add to list of bots

    # Check for collisions (with walls or other bots)
    p.performCollisionDetection()
    contacts = p.getContactPoints()
    
    num_attempts = 0

    # Any collisions? randomly choose new position
    while (len(contacts) > 0):
        # Generate new random position and rotation
        pos = [creation_region * random.random() - creation_region * 0.5, \
               creation_region * random.random() - creation_region * 0.5, 0]
        qtr = p.getQuaternionFromEuler((0, 0, random.random() * math.pi))
        
        # Set an update position and rotation
        p.resetBasePositionAndOrientation(s.index, pos, qtr)
        
        # Check for collisions for the updated position and orientation
        p.performCollisionDetection()
        contacts = p.getContactPoints()
        num_attempts += 1
        
        if num_attempts == 100:
            raise SystemExit("Too many attempts to place bots! The box is probably too small.")



# Load our simulation floor plane at the origin (0, 0, 0).
# Intentionally loaded after the bots, to not interfere with the overlap detection
floorId = p.loadURDF('simplane.urdf')

# Set friction with floor (it gets averaged with the friction of the opposing body)
p.changeDynamics(floorId, -1, lateralFriction=0.6)
p.changeDynamics(floorId, -1, spinningFriction=0.0)
#p.changeDynamics(floorId, -1, rollingFriction=0.2)

# We can check the number of bodies we have in the simulation.
#p.getNumBodies()

# Set the simulation time-step
p.setTimeStep(dt)  # 1 ms
p.setRealTimeSimulation(0)  # this is not a real-time simulation


# Set gravity
p.setGravity(0, 0, -10)


# Open log file
if len(trajectory_file_name) > 0:
    trajectory_file = open(trajectory_file_name, "w")

    
if len(video_file_name) > 0:
    video_writer = cv.VideoWriter(video_file_name, fourcc, 25, (image_size, image_size))

    
# Run the simulation for a fixed amount of steps.
for i in tqdm (range (sim_length), desc="Running simulation ..."):
    
    # Control each Smarticle
    for s_idx in range(len(smarticles)):
        s = smarticles[s_idx]
        s.OperateSmarticle(time)

    # Have the simulation engine run a step
    p.stepSimulation()

    # Keep track of time
    time += dt
    

    # Record only once every given number of frames
    if (i % record_every) == 0:

        if len(trajectory_file_name) > 0:
            # Get (base) coordinates all bots
            bots_positions = (p.getBasePositionAndOrientation(s.index)[0] for s in smarticles)

            # Write X,Y coordinates of all bots
            trajectory_file.write(' '.join('{} {}'.format(xyz[0], xyz[1]) for xyz in bots_positions))
            trajectory_file.write('\n')
        
        if len(video_file_name) > 0:
            scaled_canvas = ScaledCanvas((image_size, image_size), (box_size * -0.5, box_size * -0.5, box_size, box_size))

            for s_idx in range(len(smarticles)):
                s = smarticles[s_idx]

                for link in (-1, 0, 1):
                    if link == -1: # -1 is the main body
                        state = p.getBasePositionAndOrientation(s.index)
                    else:
                        state = p.getLinkState(s.index, link, 0, 1)

                    # Get the link's state
                    pos = state[0]
                    orientation = state[1]
                    rot_matrix = p.getMatrixFromQuaternion(orientation)

                    # Get the link's shape
                    box_shape = p.getCollisionShapeData(s.index, link)
                    box_shape = box_shape[0]
                    box_shape = box_shape[3]

                    # Define a generic box polygon
                    box_points = [[-0.5 * box_shape[0], -0.5 * box_shape[1]], \
                                  [0.5 * box_shape[0], -0.5 * box_shape[1]], \
                                  [0.5 * box_shape[0], 0.5 * box_shape[1]], \
                                  [-0.5 * box_shape[0], 0.5 * box_shape[1]]]

                    # Construct 2d rotation matrix
                    rot_matrix = [rot_matrix[0], rot_matrix[1]], [rot_matrix[3], rot_matrix[4]]
                    rot_matrix = np.transpose(rot_matrix)

                    # Rotate and shift the plotted box
                    box_points = np.dot(box_points, rot_matrix)
                    box_points = np.add(box_points, pos[0:2])

                    # Use different colors for arms and body
                    if link == -1:
                        scaled_canvas.polygon(box_points, (100,100,100))
                    elif link == 0:
                        scaled_canvas.polygon(box_points, (100,255,100))
                    else:
                        scaled_canvas.polygon(box_points, (100,100,255))

            # Convert image color for video and write it
            frame = cv.cvtColor(np.array(scaled_canvas.image), cv.COLOR_RGB2BGR)
            video_writer.write(frame)

# Close trajectory file
if len(trajectory_file_name) > 0:
    trajectory_file.close()

# Close video file
if len(video_file_name) > 0:
    video_writer.release()



Running simulation ...:   0%|          | 0/1000 [00:00<?, ?it/s]